In [ ]:
!pip install git+https://github.com/shuiruge/neural-ode.git@master

  Cloning https://github.com/shuiruge/neural-ode.git (to revision master) to /tmp/pip-req-build-l0ogcens
  Running command git clone -q https://github.com/shuiruge/neural-ode.git /tmp/pip-req-build-l0ogcens
  Created wheel for node: filename=node-0.1.0-cp36-none-any.whl size=37076 sha256=957310ad83e3b1dd87c222069289840cfc3b711b6ad804e595610474ffa91a5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5nycoqq3/wheels/36/41/e1/1cf7fd120543ff07c299bee3a2ce3fe659795c54f7e03fe9b6
Successfully built node


In [ ]:
import numpy as np
import tensorflow as tf

from node.hopfield import ContinuousTimeHopfieldLayer
                           
# for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

tf.keras.backend.clear_session()


IMAGE_SIZE = (28, 28)


def pooling(x, size):
  # x shape: [None, width, height]
  x = tf.expand_dims(x, axis=-1)
  x = tf.image.resize(x, size)
  return x  # shape: [None, size[0], size[1], 1]


def process_data(X, y, image_size):
  X = pooling(X, image_size)
  X = X / 255.
  X = tf.where(X < 0.5, -1., 1.)
  X = tf.reshape(X, [-1, image_size[0] * image_size[1]])
  y = tf.one_hot(y, 10)
  return tf.cast(X, tf.float32), tf.cast(y, tf.float32)


def get_benchmark_model(model):
  layers = [
    layer for layer in model.layers
    if not isinstance(layer, ContinuousTimeHopfieldLayer)]
  return tf.keras.Sequential(layers)


model = tf.keras.Sequential([
  tf.keras.Input([IMAGE_SIZE[0] * IMAGE_SIZE[1]]),
  tf.keras.layers.LayerNormalization(),
  tf.keras.layers.Dense(1024, activation='relu'),
  tf.keras.layers.Dense(512, activation='tanh'),
  ContinuousTimeHopfieldLayer(reg_factor=1, relax_tol=1e-3),
  tf.keras.layers.Dense(256, activation='tanh'),
  ContinuousTimeHopfieldLayer(reg_factor=1, relax_tol=1e-3),
  tf.keras.layers.Dense(128, activation='tanh'),
  ContinuousTimeHopfieldLayer(reg_factor=1, relax_tol=1e-3),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax'),
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

mnist = tf.keras.datasets.mnist
(x_train, y_train), _ = mnist.load_data()
x_train, y_train = process_data(x_train, y_train, IMAGE_SIZE)
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(1000).repeat(50).batch(128)
model.fit(dataset)

23438/23438 [==============================] - 87s 4ms/step - loss: 0.0535 - acc: 0.9921


Exame the noise effect:

In [ ]:
FLIP_RATIO = 0.3

benchmark_model = get_benchmark_model(model)

def get_hopfield_layer_ids(model):
  return [
    i for i, layer in enumerate(model.layers)
    if isinstance(layer, ContinuousTimeHopfieldLayer)]

hid = get_hopfield_layer_ids(model)[-1]

X = tf.convert_to_tensor(x_train[-2000:])
targets = tf.argmax(tf.convert_to_tensor(y_train[-2000:]), axis=1)
noised_X = tf.where(tf.random.uniform(shape=X.shape) < FLIP_RATIO,
                    -X, X)
unoised_y = tf.argmax(model.predict(X), axis=1)
y = tf.argmax(model.predict(noised_X), axis=1)
yb = tf.argmax(benchmark_model.predict(noised_X), axis=1)

sub_model = tf.keras.Sequential(model.layers[:(hid + 1)])
sy1 = sub_model.predict(X)
sy2 = sub_model.predict(noised_X)

ssub_model = tf.keras.Sequential(model.layers[:hid])
ssy1 = ssub_model.predict(X)
ssy2 = ssub_model.predict(noised_X)

def get_error_ratio(original, noised, threshold=0.2):
  diff = original - noised
  ratio_per_sample = tf.reduce_mean(
    tf.cast(tf.abs(diff) > threshold, tf.float32), axis=1)
  return ratio_per_sample

num_misleading = 0
num_corrected = 0
num_uncorrected = 0
for i, (err0, err1, ybi, yi, uyi, ti) in enumerate(zip(
    get_error_ratio(ssy1, ssy2).numpy(),
    get_error_ratio(sy1, sy2).numpy(),
    yb, y, unoised_y, targets)):
  if yi == uyi and ybi != yi:
    num_corrected += 1
  elif ybi == uyi and ybi != yi:
    num_misleading += 1
  elif yi != uyi and ybi == yi:
    num_uncorrected += 1
  else:
    pass
  if i < 50:
    print(f'{err0:.5f} => {err1:.5f} | {ybi} => {yi} | {uyi} ({ti})')
print(
  f'misleading: {num_misleading / X.shape[0]}',
  f'corrected: {num_corrected / X.shape[0]}',
  f'uncorrected: {num_uncorrected / X.shape[0]}')

0.80469 => 0.69531 | 2 => 9 | 2 (2)
0.80469 => 0.69531 | 1 => 9 | 2 (3)
0.80469 => 0.69531 | 0 => 2 | 9 (0)
0.80469 => 0.69531 | 7 => 2 | 9 (4)
0.80469 => 0.69531 | 1 => 2 | 9 (9)
0.00000 => 0.00000 | 1 => 9 | 9 (5)
0.80469 => 0.69531 | 1 => 2 | 9 (9)
0.80469 => 0.69531 | 1 => 9 | 2 (6)
0.00000 => 0.00000 | 5 => 9 | 9 (1)
0.00000 => 0.00000 | 7 => 9 | 9 (7)
0.80469 => 0.69531 | 1 => 9 | 2 (6)
0.80469 => 0.69531 | 5 => 9 | 2 (8)
0.00000 => 0.00000 | 1 => 9 | 9 (0)
0.00000 => 0.00000 | 1 => 9 | 9 (9)
0.80469 => 0.69531 | 9 => 2 | 9 (0)
0.00000 => 0.00000 | 0 => 2 | 2 (0)
0.00000 => 0.00000 | 2 => 2 | 2 (6)
0.00000 => 0.00000 | 1 => 9 | 9 (1)
0.80469 => 0.69531 | 1 => 9 | 2 (4)
0.00000 => 0.00000 | 1 => 9 | 9 (2)
0.80469 => 0.69531 | 5 => 2 | 9 (8)
0.80469 => 0.69531 | 1 => 9 | 2 (3)
0.00000 => 0.00000 | 7 => 9 | 9 (1)
0.80469 => 0.69531 | 1 => 9 | 2 (4)
0.80469 => 0.69531 | 1 => 9 | 2 (8)
0.80469 => 0.69531 | 1 => 9 | 2 (5)
0.80469 => 0.69531 | 2 => 9 | 2 (2)
0.00000 => 0.00000 | 5 => 2 

In [ ]:
for layer in model.layers:
  if isinstance(layer, ContinuousTimeHopfieldLayer):
    print(layer._stop_condition.relax_time.numpy())

80.70676
104.49519
133.97269
